## ANALYSE
Spørsmål: Kva for modell fungerer best om du vil ha færrast mogleg party killers på festen din (du vil at
modellen skal vere presis i å klassifisere nokon som ok)?
- Presisjon på klassen "ok" viser hvor presis klassifiseringene er (1.0 = 100 % riktig klassifiseringer)

### Beste presisjon i rekkefølge:
1. Decision Tree - precision for "ok": 1.00 - alle modellen klassifiserer som "ok" er "ok"
2. KNeighborsClassifier - precision for "ok": 0.56
3. Linear Regression - precision for "ok": 0.55


KNeighborsClassifier

In [ ]:
import pandas as pd
from pandas import read_csv
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

file = "machine-learning-classification/party_data.csv"
names = ["gender", "age", "study", "activity", "music", "is dancer", "ok guest"]
dataset = read_csv(file, names=names)

# Fjerner første raden for kolonnene gender, is dancer og ok guest, siden de ikke skal være inkludert som verdier i datasettet.
# Dette betyr at noen valg fjernes før one-hot-encoding
first_row = dataset.iloc[0][['gender', 'is dancer', 'ok guest']]
if (first_row == ['gender', 'is dancer', 'ok guest']).all():
    dataset = dataset.drop(index=0).reset_index(drop=True)

# Indikatorvariabler: One-hot-encoding
dummy = pd.get_dummies(dataset, columns=["age", "study", "activity", "music"])

# Ordinal Encoder: Erstatter alle verdier med tall (0 og 1 her fordi hver kolonne har to valg)
enc = OrdinalEncoder()
enc.fit(dummy) # lærer unike kategorier fra dummy dataframe
# print(enc.categories_) # skriver ut de unike kategoriene som OrdinalEncoder har lært i enc.fit(dummy), returnerer array
dataset = enc.transform(dummy) # transformerer kategoriene som ble lært i enc.fit(dummy) til 0 eller 1

# Fra numpy.ndarray til dataframe
dataset = pd.DataFrame(dataset, columns=dummy.columns)

# Splitter datasett opp i treningssett og testsett
X = dataset.drop(columns="ok guest") # Alle rader + alle kolonner utenom "ok guest"
y = dataset["ok guest"] # Alle rader + kun "ok guest" kolonnen
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

n_neighbors = [3, 5, 11, 17]
for kjøringer in n_neighbors:
    print(f"NY KJØRING/RUNDE for k = {kjøringer}")
    # KNeighborsClassifier
    neigh = KNeighborsClassifier(n_neighbors=kjøringer)

    # Prediksjoner
    neigh.fit(X_train, Y_train) # trening av modellen
    pred1 = neigh.predict(X_train)
    pred2 = neigh.predict(X_validation)

    # Accuracy
    train = accuracy_score(Y_train, pred1)
    val = accuracy_score(Y_validation, pred2)
    print(f"Trenings accuracy: {train}")
    print(f"Testing accuracy: {val}")

    # Forvekslingsmatrise: Confusion matrix
    print("Forvekslingsmatrise for treningssett:\n", confusion_matrix(Y_train, pred1))
    print("Forvekslingsmatrise for testsett:\n", confusion_matrix(Y_validation, pred2))

    # Rapport: gir bl.a presisjon i klassifisering av gjester som ok
    print("Rapport på testsettet (gir presisjon osv.)\n", classification_report(Y_validation, pred2, target_names=["not ok", "ok"]))

    # Forskjell på accuracy mellom treningssett og testsett
    print("Forskjell på accuracy mellom treningssett og testsett:", train - val, "\n")

Logistic Regression

In [ ]:
import pandas as pd
from pandas import read_csv
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

file = "machine-learning-classification/party_data.csv"
names = ["gender", "age", "study", "activity", "music", "is dancer", "ok guest"]
dataset = read_csv(file, names=names)

# Fjerner første raden for kolonnene gender, is dancer og ok guest, siden de ikke skal være inkludert som verdier i datasettet.
# Dette betyr at noen valg fjernes før one-hot-encoding
first_row = dataset.iloc[0][['gender', 'is dancer', 'ok guest']]
if (first_row == ['gender', 'is dancer', 'ok guest']).all():
    dataset = dataset.drop(index=0).reset_index(drop=True)

# Indikatorvariabler: One-hot-encoding
dummy = pd.get_dummies(dataset, columns=["age", "study", "activity", "music"], drop_first=True)

# Ordinal Encoder: Erstatter alle verdier med tall (0 og 1 her fordi hver kolonne har to valg)
enc = OrdinalEncoder()
enc.fit(dummy) # lærer unike kategorier fra dummy dataframe
# print(enc.categories_) # skriver ut de unike kategoriene som OrdinalEncoder har lært i enc.fit(dummy), returnerer array
dataset = enc.transform(dummy) # transformerer kategoriene som ble lært i enc.fit(dummy) til 0 eller 1

# Fra numpy.ndarray til dataframe
dataset = pd.DataFrame(dataset, columns=dummy.columns)

# Splitter datasett opp i treningssett og testsett
X = dataset.drop(columns="ok guest") # Alle rader + alle kolonner utenom "ok guest"
y = dataset["ok guest"] # Alle rader + kun "ok guest" kolonnen
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

penalty = ['l2', None]
for kjøringer in penalty:
    print(f"NY KJØRING/RUNDE for penalty = {kjøringer}")
    # LogisticRegression
    log = LogisticRegression(penalty=kjøringer) # lbfgs er default solver

    # Prediksjoner
    log.fit(X_train, Y_train) # trening av modellen
    pred1 = log.predict(X_train)
    pred2 = log.predict(X_validation)

    # Accuracy
    train = accuracy_score(Y_train, pred1)
    val = accuracy_score(Y_validation, pred2)
    print(f"Trenings accuracy: {train}")
    print(f"Testing accuracy: {val}")

    # Forvekslingsmatrise: Confusion matrix
    print("Forvekslingsmatrise for treningssett:\n", confusion_matrix(Y_train, pred1))
    print("Forvekslingsmatrise for testsett:\n", confusion_matrix(Y_validation, pred2))

    # Rapport: gir bl.a presisjon i klassifisering av gjester som ok
    print("Rapport på testsettet (gir presisjon osv.)\n", classification_report(Y_validation, pred2, target_names=["not ok", "ok"]))

    # Forskjell på accuracy mellom treningssett og testsett
    print("Forskjell på accuracy mellom treningssett og testsett:", train - val, "\n")

Decision Tree

In [ ]:
import pandas as pd
from pandas import read_csv
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

file = "machine-learning-classification/party_data.csv"
names = ["gender", "age", "study", "activity", "music", "is dancer", "ok guest"]
dataset = read_csv(file, names=names)

# Fjerner første raden for kolonnene gender, is dancer og ok guest, siden de ikke skal være inkludert som verdier i datasettet.
# Dette betyr at noen valg fjernes før one-hot-encoding
first_row = dataset.iloc[0][['gender', 'is dancer', 'ok guest']]
if (first_row == ['gender', 'is dancer', 'ok guest']).all():
    dataset = dataset.drop(index=0).reset_index(drop=True)

# Indikatorvariabler: One-hot-encoding
dummy = pd.get_dummies(dataset, columns=["age", "study", "activity", "music"])

# Ordinal Encoder: Erstatter alle verdier med tall (0 og 1 her fordi hver kolonne har to valg)
enc = OrdinalEncoder()
enc.fit(dummy) # lærer unike kategorier fra dummy dataframe
# print(enc.categories_) # skriver ut de unike kategoriene som OrdinalEncoder har lært i enc.fit(dummy), returnerer array
dataset = enc.transform(dummy) # transformerer kategoriene som ble lært i enc.fit(dummy) til 0 eller 1

# Fra numpy.ndarray til dataframe
dataset = pd.DataFrame(dataset, columns=dummy.columns)

# Splitter datasett opp i treningssett og testsett
X = dataset.drop(columns="ok guest") # Alle rader + alle kolonner utenom "ok guest"
y = dataset["ok guest"] # Alle rader + kun "ok guest" kolonnen
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

kriterie = ["gini", "entropy"]
for kjøringer in kriterie:
    print(f"NY KJØRING/RUNDE for criterion = {kjøringer}")
    # DecisionTreeClassifier
    tree = DecisionTreeClassifier(criterion=kjøringer)

    # Prediksjoner
    tree.fit(X_train, Y_train) # trening av modellen
    pred1 = tree.predict(X_train)
    pred2 = tree.predict(X_validation)

    # Accuracy
    train = accuracy_score(Y_train, pred1)
    val = accuracy_score(Y_validation, pred2)
    print(f"Trenings accuracy: {train}")
    print(f"Testing accuracy: {val}")

    # Forvekslingsmatrise: Confusion matrix
    print("Forvekslingsmatrise for treningssett:\n", confusion_matrix(Y_train, pred1))
    print("Forvekslingsmatrise for testsett:\n", confusion_matrix(Y_validation, pred2))

    # Rapport: gir bl.a presisjon i klassifisering av gjester som ok
    print("Rapport på testsettet (gir presisjon osv.)\n", classification_report(Y_validation, pred2, target_names=["not ok", "ok"]))

    # Forskjell på accuracy mellom treningssett og testsett
    print("Forskjell på accuracy mellom treningssett og testsett:", train - val, "\n")